In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

In [ ]:
#Drop EIN and Special_considerations
application_df = application_df.drop(columns=["EIN", "SPECIAL_CONSIDERATIONS"])
application_df

In [ ]:
# Look at Name value counts for binning
v_counts = application_df["NAME"].value_counts()
v_counts

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# cut out the names of companies that have less instances/cut out outliers
Names_to_replace = list(v_counts[v_counts<10].index)

In [ ]:
# Replace in dataframe
for name in Names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"Other")

# Check to make sure binning was successful
application_df['NAME'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
bin_classifier = application_df["CLASSIFICATION"].value_counts()
bin_classifier

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
bin_classification = bin_classifier[bin_classifier>1]
bin_classification

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace and up the amount to 100
classifications_to_replace = list(bin_classifier[bin_classifier<100].index)
classifications_to_replace

In [ ]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, dtype=float)
application_df.head(10)

In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop("IS_SUCCESSFUL", axis=1).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
hidden_layer_nodes1 = 10
hidden_layer_nodes2 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_nodes1, input_dim=input_features, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_nodes2, input_dim=input_features, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [ ]:
# Train the model
model_fit = nn.fit(X_train_scaled, y_train, epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
nn.save("AlphabetSoupCharity_Optimization.h5")